In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

la liste de tous les départements de l'occitanie

In [ ]:
dep_occitanie=["/trouver-un-professionnel-de-sante/chirurgiens-dentistes/09-ariege",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/11-aude","/trouver-un-professionnel-de-sante/chirurgiens-dentistes/12-aveyron","/trouver-un-professionnel-de-sante/chirurgiens-dentistes/30-gard",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/31-haute-garonne","/trouver-un-professionnel-de-sante/chirurgiens-dentistes/32-gers","/trouver-un-professionnel-de-sante/chirurgiens-dentistes/34-herault",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/46-lot",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/48-lozere",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/65-hautes-pyrenees",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/66-pyrenees-orientales",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/81-tarn",
"/trouver-un-professionnel-de-sante/chirurgiens-dentistes/82-tarn-et-garonne"]

In [ ]:
base_url = "http://annuairesante.ameli.fr"
url = "http://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/chirurgiens-dentistes/{dep}-loire-atlantique"
#url à remplacer avec les départements de l'occitanie

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
a_href = soup.findAll('a')

In [ ]:
list_url_ville = [] 
list_ville = [] 
a_href = a_href[3:-6]
for link in a_href:
    if link.has_attr('href'):
        list_url_ville.append(link['href'])
        list_ville.append(link.text)

In [ ]:
def get_praticien_ville(ville, url):
    response_name = requests.get(base_url + url)
    soup_name = BeautifulSoup(response_name.text, 'html.parser')
    name = soup_name.findAll('a')
    adress = soup_name.findAll(class_="item left adresse")
    
    # contient le nom et le prenom de tous les praticiens de la ville
    name_clean = [] 
    for n in name:
        if n.strong:
            name_clean.append(n)
            
    # liste des informations des praticiens de la ville: nom, adresse et ville
    list_info_praticiens = [] 
    for j in range(len(name_clean)):
        
        info_praticiens = {}        
        info_praticiens['nom'] = name_clean[j].text
        info_praticiens['adress'] = adress[j].text
        info_praticiens['ville'] = ville
        list_info_praticiens.append(info_praticiens)
        
    return list_info_praticiens


In [ ]:
all_praticiens = [] 
for i in range(len(list_url_ville)):
    praticiens = get_praticien_ville(list_ville[i], list_url_ville[i])
    all_praticiens.extend(praticiens)     
# all_praticiens

In [ ]:
df = pd.DataFrame(all_praticiens)

In [ ]:
df.shape

In [ ]:
df['code_postal'] = df.ville.apply(lambda x: re.findall('\d+', x)[0])

In [ ]:
groupby_code_postal = df.groupby(['code_postal']).size().reset_index()
groupby_code_postal.columns=['code_postal','nb_praticiens']
groupby_code_postal

In [ ]:
code_commune = pd.read_csv("donnees_communes.csv", sep=";")

In [1]:
code_commune_post = pd.read_csv("laposte_hexasmal.csv", sep=";")

NameError: name 'pd' is not defined

In [ ]:
code_commune.CODCOM=code_commune.CODCOM.apply(lambda x : str(x).zfill(3))
code_commune['code_commune_insee']=code_commune.CODDEP+code_commune.CODCOM

In [ ]:
merges=pd.merge(code_commune_post,code_commune,on='code_commune_insee')
merges.code_postal=merges.code_postal.astype(str)

In [ ]:
merges=merges[["code_postal","PTOT"]]
merges

In [ ]:
newDataFrame=groupby_code_postal.merge(merges,on ="code_postal",how='inner')
newDataFrame['population_sur_nb_praticiens']=newDataFrame.PTOT/newDataFrame.nb_praticiens
newDataFrame

In [ ]:
# les statistiques
newDataFrame.describe()

In [ ]:
y = pd.Series(newDataFrame.PTOT) 
x = pd.Series(newDataFrame.nb_praticiens) 
plt.scatter(x, y) 
plt.title("corrélation du nombre d'habitant de chaque ville en fonction du nombre de praticien")
plt.xlabel('x: nombre de praticiens par commune') 
plt.ylabel('y: la population total par commune')
  
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1)) 
         (np.unique(x)), color='red') 